**list of contents**

[1. Preparation](#Preparation)<br>
&nbsp;[1.1 import lib](#import-lib)<br>
&nbsp;[1.2 data import](#data-import)<br>
&nbsp;[1.3 data preparation](#data-import)<br>
&ensp;[1.3.1 data diagram](#data-diagram)<br>
&ensp;[1.3.2 check data shape](#check-data-shape)<br>
&nbsp;[1.4 reshape data](#reshape-data)<br>
[2. ML Training](#ML-Training)<br>
&nbsp;[2.1 ML data preparation](#ML-data-preparation)<br>
&ensp;[2.1.1 select data and rename columns](#select-data-and-rename-columns)<br>
&ensp;[2.1.2 merge dataframe and set key](#merge-dataframe-and-set-key)<br>
&nbsp;[2.2 Training](#Training)<br>
&ensp;[2.2.1 Position Estimation FFNN](#Position-Estimation-FFNN)<br>
&emsp;[2.2.1.1 FFNN Regression here start](#FFNN-Regression-here-start)<br>
&ensp;[2.2.2 Position Classification FFNN](#Position-Classification-FFNN)<br>
&emsp;[2.2.2.1 FFNN Classification here start](#FFNN-Classification-here-start)<br>
&ensp;[2.2.3 Position Estimation CNN](#Position-Estimation-CNN)<br>
&emsp;[2.2.3.1 CNN Regression here start](#CNN-Regression-here-start)<br>

# Preparation

## import lib

In [ ]:
# import lib
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Input, GlobalAveragePooling1D, Dropout
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import normalize, LabelBinarizer
from sklearn.model_selection import train_test_split
import keras_tuner
#from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor
#from keras.utils.vis_utils import plot_model
import graphviz
import pydot
%matplotlib inline
sns.set(style="darkgrid")

In [ ]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))
# [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

print(tf.test.is_built_with_cuda)
# <function is_built_with_cuda at 0x7f4f5730fbf8>

print(tf.test.gpu_device_name())
# /device:GPU:0

print(tf.config.get_visible_devices())
# [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]



## data import

In [ ]:
# define transform function
def umform(data):
    pur_data=[]
    for i in data:
        sp = i.split(' ')
        sp[:]=[x for x in sp if x]
        pur_data.append(sp)
    return pur_data

# define labels for dataframe
labels= ['ZeitInSec',
         'ZeitInNanoSec',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[0]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[1]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[2]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[3]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[4]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[5]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[6]',
         'axisTauMsr_LBR_iiwa_14_R820_1[0]',
         'axisTauMsr_LBR_iiwa_14_R820_1[1]',
         'axisTauMsr_LBR_iiwa_14_R820_1[2]',
         'axisTauMsr_LBR_iiwa_14_R820_1[3]',
         'axisTauMsr_LBR_iiwa_14_R820_1[4]',
         'axisTauMsr_LBR_iiwa_14_R820_1[5]',
         'axisTauMsr_LBR_iiwa_14_R820_1[6]',
         'cartForce1_X',
         'cartForceVar1_X',
         'cartForce1_Y',
         'cartForceVar1_Y',
         'cartForce1_Z',
         'cartForceVar1_Z',
         'cartTorque1_TauX',
         'cartVarTorque1_TauX',
         'cartTorque1_TauY',
         'cartVarTorque1_TauY',
         'cartTorque1_TauZ',
         'cartVarTorque1_TauZ',
         'axisQMsr_LBR_iiwa_14_R820_1[0]',
         'axisQMsr_LBR_iiwa_14_R820_1[1]',
         'axisQMsr_LBR_iiwa_14_R820_1[2]',
         'axisQMsr_LBR_iiwa_14_R820_1[3]',
         'axisQMsr_LBR_iiwa_14_R820_1[4]',
         'axisQMsr_LBR_iiwa_14_R820_1[5]',
         'axisQMsr_LBR_iiwa_14_R820_1[6]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[0]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[1]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[2]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[3]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[4]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[5]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[6]',
         'CartPosMsr1_X',
         'CartPosMsr1_Y',
         'CartPosMsr1_Z',
         'CartPosCmd1_X',
         'CartPosCmd1_Y',
         'CartPosCmd1_Z']

for i in range(0,11):
    for j in range(0,11):
        file = open("Data/versuch_f1/f_"+ str(i) + "_" + str(j) +"_demoRecord.txt",'r')

        cl = []
        for line in file:
            cl.append(line.strip())
        data = cl[1:]

        pur = umform(data)

        df = pd.DataFrame(pur, columns=labels)
        df = df.astype(float)
    
        locals()[f'df{i}_{j}'] = df

## Data preparation

### data diagram

#### Cart Forces TCP

In [ ]:
plt.figure(figsize=(110,110))
plt.suptitle("Forces on different points", fontsize=100)
for i in range (0,11):
    for j in range (0,11):
        plt.subplot(11, 11, i*11+j+1)
        plt.plot(locals()[f'df{i}_{j}']['cartForce1_X'],label="x")
        plt.plot(locals()[f'df{i}_{j}']['cartForce1_Y'],label="y")
        plt.plot(locals()[f'df{i}_{j}']['cartForce1_Z'],label="z")
        plt.title("df{0}_{1}".format(i,j))
        plt.ylim([-10,50])
        plt.legend()
        j += 1
    i += 1
plt.show()

#### Cart Torques TCP

In [ ]:
plt.figure(figsize=(110,110))
plt.suptitle("Torques on different points", fontsize=100)
for i in range (0,11):
    for j in range (0,11):
        plt.subplot(11, 11, i*11+j+1)
        plt.plot(locals()[f'df{i}_{j}']['cartTorque1_TauX'],label="x")
        plt.plot(locals()[f'df{i}_{j}']['cartTorque1_TauY'],label="y")
        plt.plot(locals()[f'df{i}_{j}']['cartTorque1_TauZ'],label="z")
        plt.title("df{0}_{1}".format(i,j))
        plt.ylim([-3,3])
        plt.legend()
plt.show()

#### External Torques on different joints

In [ ]:
plt.figure(figsize=(110,110))
for i in range (0,11):
    for j in range (0,11):
        plt.subplot(11, 11, i*11+j+1)
        for k in range(0,7):
            plt.plot(locals()[f'df{i}_{j}']['axisTauExtMsr_LBR_iiwa_14_R820_1[{0}]'.format(k)],label="j{0}".format(k))
        plt.title("df{0}_{1}".format(i,j))
        plt.ylim([-30,20])
        plt.legend()
plt.suptitle("External Torques on different joints",fontsize=100)
plt.show()

#### Internal Torques on different joints

In [ ]:
plt.figure(figsize=(110,110))
for i in range (0,11):
    for j in range (0,11):
        plt.subplot(11, 11, i*11+j+1)
        for k in range(0,7):
            plt.plot(locals()[f'df{i}_{j}']['axisTauMsr_LBR_iiwa_14_R820_1[{0}]'.format(k)],label="j{0}".format(k))
        plt.title("df{0}_{1}".format(i,j))
        plt.ylim([-45,85])
        plt.legend()
plt.title("Internal Torques on different joints",fontsize=100)
plt.show()

In [ ]:
plt.plot(df0_0['CartPosMsr1_Z'],label="z_ist")
plt.plot(df0_0['CartPosCmd1_Z'],label="z_soll")
plt.legend()

In [ ]:
plt.plot(df0_0['cartForce1_Z'])

### check data shape

In [ ]:
data_shape=[]
for i in range (0,11):
    for j in range (0,11):
        #if (locals()[f'df{i}_{j}'].shape[0] < 5000):
        data_shape.append(locals()[f'df{i}_{j}'].shape[0])
sns.histplot(data=data_shape)

## reshape data
leave only the last 3000 points

In [ ]:
for i in range (0,11):
    for j in range (0,11):
        locals()[f'df{i}_{j}_reshape']=locals()[f'df{i}_{j}'][-3000:].reset_index()

# ML Training

## ML data preparation

### select data and rename columns

**labels:**
| Original Name                       | Description             | Rename |
| -----                               | -----------             | ------ |
| ZeitInSec                           | Time in s               |        |
| ZeitInNanoSec                       | Time in ns              |        |
| axisTauExtMsr_LBR_iiwa_14_R820_1[0] | External Torques of A1  | exT_A1 |
| axisTauExtMsr_LBR_iiwa_14_R820_1[1] | External Torques of A2  | exT_A2 |
| axisTauExtMsr_LBR_iiwa_14_R820_1[2] | External Torques of A3  | exT_A3 |
| axisTauExtMsr_LBR_iiwa_14_R820_1[3] | External Torques of A4  | exT_A4 |
| axisTauExtMsr_LBR_iiwa_14_R820_1[4] | External Torques of A5  | exT_A5 |
| axisTauExtMsr_LBR_iiwa_14_R820_1[5] | External Torques of A6  | exT_A6 |
| axisTauExtMsr_LBR_iiwa_14_R820_1[6] | External Torques of A7  | exT_A7 |
| axisTauMsr_LBR_iiwa_14_R820_1[0]    | Measured Torques of A1  | msT_A1 |
| axisTauMsr_LBR_iiwa_14_R820_1[1]    | Measured Torques of A2  | msT_A2 |
| axisTauMsr_LBR_iiwa_14_R820_1[2]    | Measured Torques of A3  | msT_A3 |
| axisTauMsr_LBR_iiwa_14_R820_1[3]    | Measured Torques of A4  | msT_A4 |
| axisTauMsr_LBR_iiwa_14_R820_1[4]    | Measured Torques of A5  | msT_A5 |
| axisTauMsr_LBR_iiwa_14_R820_1[5]    | Measured Torques of A6  | msT_A6 |
| axisTauMsr_LBR_iiwa_14_R820_1[6]    | Measured Torques of A7  | msT_A7 |
| cartForce1_X                        | Cartesian Force along x | Fx     |
| cartForceVar1_X                     |                         |        |
| cartForce1_Y                        | Cartesian Force along y | Fy     |
| cartForceVar1_Y                     |                         |        |
| cartForce1_Z                        | Cartesian Force along z | Fz     |
| cartForceVar1_Z                     |                         |        |
| cartTorque1_TauX                    | Cartesian Torque in x   | Tx     |
| cartVarTorque1_TauX                 |                         |        |
| cartTorque1_TauY                    | Cartesian Torque in y   | Ty     |
| cartVarTorque1_TauY                 |                         |        |
| cartTorque1_TauZ                    | Cartesian Torque in z   | Tz     |
| cartVarTorque1_TauZ                 |                         |        |
| axisQMsr_LBR_iiwa_14_R820_1[0]      | Current Joint Position of A1 | cur_A1 |
| axisQMsr_LBR_iiwa_14_R820_1[1]      | Current Joint Position of A2 | cur_A2 |
| axisQMsr_LBR_iiwa_14_R820_1[2]      | Current Joint Position of A3 | cur_A3 |
| axisQMsr_LBR_iiwa_14_R820_1[3]      | Current Joint Position of A4 | cur_A4 |
| axisQMsr_LBR_iiwa_14_R820_1[4]      | Current Joint Position of A5 | cur_A5 |
| axisQMsr_LBR_iiwa_14_R820_1[5]      | Current Joint Position of A6 | cur_A6 |
| axisQMsr_LBR_iiwa_14_R820_1[6]      | Current Joint Position of A7 | cur_A7 |
| axisQCmdT1m_LBR_iiwa_14_R820_1[0]   | Commanded Joint Position of A1 | comd_A1 |
| axisQCmdT1m_LBR_iiwa_14_R820_1[1]   | Commanded Joint Position of A2 | comd_A2 |
| axisQCmdT1m_LBR_iiwa_14_R820_1[2]   | Commanded Joint Position of A3 | comd_A3 |
| axisQCmdT1m_LBR_iiwa_14_R820_1[3]   | Commanded Joint Position of A4 | comd_A4 |
| axisQCmdT1m_LBR_iiwa_14_R820_1[4]   | Commanded Joint Position of A5 | comd_A5 |
| axisQCmdT1m_LBR_iiwa_14_R820_1[5]   | Commanded Joint Position of A6 | comd_A6 |
| axisQCmdT1m_LBR_iiwa_14_R820_1[6]   | Commanded Joint Position of A7 | comd_A7 |
| CartPosMsr1_X                       | Current Cartesian Position in x | curCart_x |
| CartPosMsr1_Y                       | Current Cartesian Position in y | curCart_y |
| CartPosMsr1_Z                       | Current Cartesian Position in z | curCart_z |
| CartPosCmd1_X                       | Commanded Cartesian Position in x | comdCart_x |
| CartPosCmd1_Y                       | Commanded Cartesian Position in y | comdCart_y |
| CartPosCmd1_Z                       | Commanded Cartesian Position in z | comdCart_z |

select the needed data

In [ ]:
for i in range (0,11):
    for j in range (0,11):
        locals()[f'df{i}_{j}_select']=locals()[f'df{i}_{j}_reshape'][[
         'axisTauExtMsr_LBR_iiwa_14_R820_1[0]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[1]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[2]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[3]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[4]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[5]',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[6]',
         'axisTauMsr_LBR_iiwa_14_R820_1[0]',
         'axisTauMsr_LBR_iiwa_14_R820_1[1]',
         'axisTauMsr_LBR_iiwa_14_R820_1[2]',
         'axisTauMsr_LBR_iiwa_14_R820_1[3]',
         'axisTauMsr_LBR_iiwa_14_R820_1[4]',
         'axisTauMsr_LBR_iiwa_14_R820_1[5]',
         'axisTauMsr_LBR_iiwa_14_R820_1[6]',
         'cartForce1_X',
         'cartForce1_Y',
         'cartForce1_Z',
         'cartTorque1_TauX',
         'cartTorque1_TauY',
         'cartTorque1_TauZ',
         'axisQMsr_LBR_iiwa_14_R820_1[0]',
         'axisQMsr_LBR_iiwa_14_R820_1[1]',
         'axisQMsr_LBR_iiwa_14_R820_1[2]',
         'axisQMsr_LBR_iiwa_14_R820_1[3]',
         'axisQMsr_LBR_iiwa_14_R820_1[4]',
         'axisQMsr_LBR_iiwa_14_R820_1[5]',
         'axisQMsr_LBR_iiwa_14_R820_1[6]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[0]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[1]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[2]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[3]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[4]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[5]',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[6]',
         'CartPosMsr1_X',
         'CartPosMsr1_Y',
         'CartPosMsr1_Z',
         'CartPosCmd1_X',
         'CartPosCmd1_Y',
         'CartPosCmd1_Z']]
                                                                      

rename the columns

In [ ]:
for i in range (0,11):
    for j in range (0,11):
        locals()[f'df{i}_{j}_rename']=locals()[f'df{i}_{j}_select'].rename(columns={
         'axisTauExtMsr_LBR_iiwa_14_R820_1[0]':'exT_A1',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[1]':'exT_A2',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[2]':'exT_A3',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[3]':'exT_A4',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[4]':'exT_A5',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[5]':'exT_A6',
         'axisTauExtMsr_LBR_iiwa_14_R820_1[6]':'exT_A7',
         'axisTauMsr_LBR_iiwa_14_R820_1[0]':'msT_A1',
         'axisTauMsr_LBR_iiwa_14_R820_1[1]':'msT_A2',
         'axisTauMsr_LBR_iiwa_14_R820_1[2]':'msT_A3',
         'axisTauMsr_LBR_iiwa_14_R820_1[3]':'msT_A4',
         'axisTauMsr_LBR_iiwa_14_R820_1[4]':'msT_A5',
         'axisTauMsr_LBR_iiwa_14_R820_1[5]':'msT_A6',
         'axisTauMsr_LBR_iiwa_14_R820_1[6]':'msT_A7',
         'cartForce1_X':'Fx',
         'cartForce1_Y':'Fy',
         'cartForce1_Z':'Fz',
         'cartTorque1_TauX':'Tx',
         'cartTorque1_TauY':'Ty',
         'cartTorque1_TauZ':'Tz',
         'axisQMsr_LBR_iiwa_14_R820_1[0]':'cur_A1',
         'axisQMsr_LBR_iiwa_14_R820_1[1]':'cur_A2',
         'axisQMsr_LBR_iiwa_14_R820_1[2]':'cur_A3',
         'axisQMsr_LBR_iiwa_14_R820_1[3]':'cur_A4',
         'axisQMsr_LBR_iiwa_14_R820_1[4]':'cur_A5',
         'axisQMsr_LBR_iiwa_14_R820_1[5]':'cur_A6',
         'axisQMsr_LBR_iiwa_14_R820_1[6]':'cur_A7',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[0]':'comd_A1',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[1]':'comd_A2',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[2]':'comd_A3',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[3]':'comd_A4',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[4]':'comd_A5',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[5]':'comd_A6',
         'axisQCmdT1m_LBR_iiwa_14_R820_1[6]':'comd_A7',
         'CartPosMsr1_X':'curCart_x',
         'CartPosMsr1_Y':'curCart_y',
         'CartPosMsr1_Z':'curCart_z',
         'CartPosCmd1_X':'comdCart_x',
         'CartPosCmd1_Y':'comdCart_y',
         'CartPosCmd1_Z':'comdCart_z'})

In [ ]:
df0_0_rename

### merge dataframe and set key

first add key for each dataframe

In [ ]:
for i in range (0,11):
    for j in range (0,11):
        locals()[f'df{i}_{j}_key']=locals()[f'df{i}_{j}_rename']
        locals()[f'df{i}_{j}_key']['key']="{0},{1}".format(i,j)

In [ ]:
df0_0_key

merge all dataframe into one and save for later use

In [ ]:
full_df = pd.DataFrame()
for i in range (0,11):
    for j in range (0,11):
        full_df = pd.concat([full_df,locals()[f'df{i}_{j}_key']],ignore_index=True)

In [ ]:
full_df

In [ ]:
full_df.to_csv('full.csv', index=False)

## Training

### Position Estimation FFNN

#### FFNN Regression here start

**import the saved csv file**

In [ ]:
data_df = pd.read_feather('../data/feather/versuch_f1_3000_1.feather')

**transform cartesian position**

since the data is recorded according to world frame, the cartesian position of the TCP have values over hundert. Here we will transform it according to the center position of the plate.

In [ ]:
def transform_cart_pos(dataFrame):
    new_df = dataFrame.copy()
    new_df = new_df
    # x
    new_df['curCart_x'] = new_df['curCart_x'] - 578.6
    new_df['comdCart_x'] = new_df['comdCart_x'] - 578.6
    #y
    new_df['curCart_y'] = new_df['curCart_y'] + 261.4
    new_df['comdCart_y'] = new_df['comdCart_y'] + 261.4
    #z
    new_df['curCart_z'] = new_df['curCart_z'] - 375
    new_df['comdCart_z'] = new_df['comdCart_z'] - 375
    return new_df

In [ ]:
df_transform = transform_cart_pos(data_df)

shuffle the data

In [ ]:
df_shuffle = df_transform.sample(frac=1, random_state=1)
X = df_shuffle[['exT_A1','exT_A2','exT_A3','exT_A4','exT_A5','exT_A6','exT_A7',
                'msT_A1','msT_A2','msT_A3','msT_A4','msT_A5','msT_A6','msT_A7',
                'Fx','Fy','Fz','Tx','Ty','Tz']].to_numpy()
y = df_shuffle[['curCart_x','curCart_y','curCart_z']].to_numpy()

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=3)

In [ ]:
X_train.min(),X_train.max()

In [ ]:
def build_model(hp):
    # create model
    model = Sequential()
    model.add(Dense(20, input_shape=(20,), activation='relu'))
    # Tune number of layers
    for i in range(hp.Int("num_layers", 1, 5)):
        model.add(Dense(
                # Tune number of units
                units = hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                # Tune activation function
                activation = hp.Choice("activation", ["relu","tanh"])
                        )
                 )
    # Tune whether to use dropout
    #if hp.Boolean("dropout"):
        #model.add(Dropout(rate=0.2))
    model.add(Dense(3, activation='linear'))
    # definr the optimizer learning rate as hyperparameter
    lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    # compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae"])
    return model

In [ ]:
hp = keras_tuner.HyperParameters()
build_model(hp)

tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_mae",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="test_mae2",
    max_consecutive_failed_trials=3,
)

tuner.search(X_train, y_train, epochs=30, validation_data=(X_val, y_val),
            callbacks=[keras.callbacks.TensorBoard('my_dir/tb_mae2')])

In [ ]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.build(input_shape=(20,))
best_model.summary()

In [ ]:
tuner.oracle.get_best_trials()[0].trial_id

In [ ]:
def baseline_model(hidden_layer_dim):
    # create model
    model = Sequential()
    model.add(Dense(20, input_shape=(20,), kernel_initializer='normal', activation='relu'))
    model.add(Dense(hidden_layer_dim, activation='relu'))
    model.add(Dense(3, kernel_initializer='normal', activation='linear'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    return model

In [ ]:
model = KerasRegressor(baseline_model, hidden_layer_dim=20)

In [ ]:
history = test_model.fit(X_train, y_train, epochs=30, batch_size=30,  verbose=1, validation_data=[X_val, y_val])

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=30,  verbose=1, validation_data=[X_val, y_val])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss (mse)')
plt.ylabel('loss[mm]')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('model mae')
plt.ylabel('mae[mm]')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
pred = model.predict(X_test)
diff = pred-y_test

plt.boxplot(diff)
plt.ylabel('Error [mm]')

### Position Classification FFNN

<br>
<img src="./pic/CubePlatte_Classification.drawio.png" width="400">
<br>

In [ ]:
# define position
def add_pos_label(dataFrame):
    for i in range(0,dataFrame.shape[0]):
        x = dataFrame['curCart_x'][i]
        y = dataFrame['curCart_y'][i]
        f1 = y - 2*x
        f2 = y + 2*x
        f3 = y - x/2
        f4 = y + x/2
        # class 1
        if (f1<0) & (f3>=0):
            dataFrame.loc[i,'position']='class 1'
        # class 2
        elif (f3<0) & (f4>=0):
            dataFrame.loc[i,'position']='class 2'
        # class 3
        elif (f4<0) & (f2>=0):
            dataFrame.loc[i,'position']='class 3'
        # class 4
        elif (f2<0) & (f1<=0):
            dataFrame.loc[i,'position']='class 4'
        # class 5
        elif (f1>0) & (f3<=0):
            dataFrame.loc[i,'position']='class 5'
        # class 6
        elif (f3>0) & (f4<=0):
            dataFrame.loc[i,'position']='class 6'
        # class 7
        elif (f4>0) & (f2<=0):
            dataFrame.loc[i,'position']='class 7'
        # class 8
        elif (f2>0) & (f1>=0):
            dataFrame.loc[i,'position']='class 8'
    return dataFrame

In [ ]:
df_pos = add_pos_label(df_transform)

In [ ]:
# save for later use
df_pos.to_csv('df_pos.csv', index=False)

#### FFNN Classification here start

In [ ]:
# import csv
df_pos = pd.read_csv('df_pos.csv')

In [ ]:
df_pos

In [ ]:
X_cl_ffnn = df_pos[['exT_A1','exT_A2','exT_A3','exT_A4','exT_A5','exT_A6','exT_A7',
                'msT_A1','msT_A2','msT_A3','msT_A4','msT_A5','msT_A6','msT_A7',
                'Fx','Fy','Fz','Tx','Ty','Tz']].to_numpy()
y_cl_ffnn = df_pos[['position']].to_numpy()

In [ ]:
binarizer = LabelBinarizer().fit(y_cl_ffnn)
y_label = binarizer.transform(y_cl_ffnn)

In [ ]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X_cl_ffnn, y_label, test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=3)

In [ ]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):
        # create model
        model = Sequential()
        model.add(Dense(20, input_shape=(20,), activation='relu'))
        # Tune number of layers
        for i in range(hp.Int("num_layers", 1, 5)):
            model.add(Dense(
                    # Tune number of units
                    units = hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                    # Tune activation function
                    activation = hp.Choice("activation", ["relu","tanh"])
                            )
                     )
        # Tune whether to use dropout
        #if hp.Boolean("dropout"):
            #model.add(Dropout(rate=0.2))
        model.add(Dense(8, activation='softmax'))
        # definr the optimizer learning rate as hyperparameter
        lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        # compile model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                      loss="categorical_crossentropy", 
                      metrics=["accuracy"])
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [16, 32]),
            **kwargs,
        )

tuner_cl = keras_tuner.RandomSearch(
    MyHyperModel(),
    objective="val_accuracy",
    max_trials=5,
    overwrite=True,
    directory="my_dir",
    project_name="test_cl2",
)

In [ ]:
tuner_cl.search(X_train, y_train, epochs=20, validation_data=(X_val, y_val),
            callbacks=[keras.callbacks.TensorBoard('my_dir/tb_cl2')])

In [ ]:
models_cl = tuner_cl.get_best_models(num_models=2)
best_cl_model = models_cl[0]
best_cl_model.build(input_shape=(20,))
best_cl_model.summary()

In [ ]:
best_cl_model.save('model_cl2')

In [ ]:
test_cl_model = keras.models.load_model('model_cl')

In [ ]:
plot_model(test_cl_model, show_shapes=True, show_layer_names=True)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "my_dir/tb_cl"

In [ ]:
score, acc = test_cl_model.evaluate(X_test, y_test) #batch_size=batch_size
print('Test score:', score)
print('Test accuracy:', acc)

### Position Estimation CNN

In [ ]:
# import csv
df_pos = pd.read_csv('df_pos.csv')

In [ ]:
select_df = df_pos[['exT_A1','exT_A2','exT_A3','exT_A4','exT_A5','exT_A6','exT_A7',
                'msT_A1','msT_A2','msT_A3','msT_A4','msT_A5','msT_A6','msT_A7',
                'Fx','Fy','Fz','Tx','Ty','Tz','curCart_x','curCart_y','curCart_z','key','position']]

transform in time series

In [ ]:
def time_series_transform(dataframe):
    test_df = timeseries_dataset_from_array(dataframe, None, sequence_length=10,shuffle=False)
    el = []
    for element in test_df:
        el.append(element.numpy())
    el_res = np.concatenate(el, axis=0)
    return el_res

In [ ]:
for i in range(0,11):
    for j in range(0,11):
        locals()[f'key{i}_{j}'] = select_df.loc[select_df['key'] == '{0},{1}'.format(i,j)]
        for k in range(0,20):
            locals()[f'key{i}_{j}_{k}'] = time_series_transform(locals()[f'key{i}_{j}'].iloc[:,k])

In [ ]:
for i in range(0,11):
    for j in range(0,11):
        locals()[f'ts_key{i}_{j}'] = []
        for k in range(0,20):
            locals()[f'ts_key{i}_{j}'].append(locals()[f'key{i}_{j}_{k}'])
        locals()[f'ts_key{i}_{j}'] = np.stack(locals()[f'ts_key{i}_{j}'], axis=-1)
ts_input = np.stack(ts_key0_0, axis=-1)

In [ ]:
ts_input = []
for i in range(0,11):
    for j in range(0,11):
        ts_input.append(locals()[f'ts_key{i}_{j}'])
ts_input = np.concatenate(ts_input, axis=0)

reshape output

In [ ]:
ts_output = []
for i in range(0,11):
    for j in range(0,11):
        ts_output.append(locals()[f'key{i}_{j}'][['curCart_x','curCart_y']][9:].to_numpy())
ts_output = np.concatenate(ts_output, axis=0)

In [ ]:
ts_output_pos = []
for i in range(0,11):
    for j in range(0,11):
        ts_output_pos.append(locals()[f'key{i}_{j}'][['position']][9:].to_numpy())
ts_output_pos = np.concatenate(ts_output_pos, axis=0)

save for later use

In [ ]:
np.savez_compressed('ts_data', ts_input=ts_input,ts_output=ts_output,ts_output_pos=ts_output_pos)

#### CNN Regression here start

import saved preprocessed data

In [ ]:
ts_data = np.load('./ts_data.npz', allow_pickle=True)

ts_input = ts_data['ts_input']
ts_output = ts_data['ts_output']
ts_output_pos = ts_data['ts_output_pos']

In [ ]:
# train test split
X_train_full, X_test, y_train_full, y_test = train_test_split(ts_input, ts_output, test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=3)

In [ ]:
def build_cnn_model(hp):
    # create model
    model = Sequential()
    
    # adding input convolutional layer
    model.add(Conv1D(
                # input shape
                input_shape = (10,20),
                # filters
                filters = hp.Int('conv_input_filter', min_value=16, max_value=128, step=16),
                # kernel_size
                kernel_size = hp.Choice('conv_input_kernel', values=[3,5]),
                # activation
                activation = hp.Choice('conv_input_activation', ['relu','tanh']),
                padding="same"
              ))
    
    # adding convolutional layer
    for i in range (hp.Int("num_layers",1,6)):
        model.add(Conv1D(
                    # filters
                    filters = hp.Int(f'conv_{i}_filter', min_value=16, max_value=128, step=16),
                    # kernel_size
                    kernel_size = hp.Choice(f'conv_{i}_kernel', values=[3,5]),
                    # activation
                    activation = hp.Choice(f'conv_{i}_activation', ['relu','tanh']),
                    padding="same"
                ))
    
    # Tune whether to use dropout
    #if hp.Boolean("dropout"):
        #model.add(Dropout(rate=0.25))
    
    # pooling layer
    model.add(GlobalAveragePooling1D())
    
    # output layer
    model.add(Dense(2, activation='linear'))
    
    # definr the optimizer learning rate as hyperparameter
    lr = hp.Float("lr", min_value=1e-3, max_value=1e-2, sampling="log")
    
    # compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                  loss="mse", 
                  metrics=["mae"])
    return model

In [ ]:
hp_cnn = keras_tuner.HyperParameters()
build_cnn_model(hp_cnn)

tuner_cnn = keras_tuner.RandomSearch(
    hypermodel=build_cnn_model,
    objective="val_mae",
    max_trials=5,
    #executions_per_trial=3,
    overwrite=True,
    directory="mydir",
    project_name="test_cnn",
)

tuner_cnn.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val),
                 callbacks=[keras.callbacks.TensorBoard('my_dir/tb_cnn')])

#### CNN Classification here start

In [ ]:
ts_data = np.load('./ts_data.npz', allow_pickle=True)

ts_input = ts_data['ts_input']
ts_output = ts_data['ts_output']
ts_output_pos = ts_data['ts_output_pos']

In [ ]:
binarizer = LabelBinarizer().fit(ts_output_pos)
y_label = binarizer.transform(ts_output_pos)

In [ ]:
# train test split
X_train_full, X_test, y_train_full, y_test = train_test_split(ts_input, y_label, test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=3)

In [ ]:
def build_cnn_cl_model(hp):
    # create model
    model = Sequential()
    
    # adding input convolutional layer
    model.add(Conv1D(
                # input shape
                input_shape = (10,20),
                # filters
                filters = hp.Int('conv_input_filter', min_value=16, max_value=128, step=16),
                # kernel_size
                kernel_size = hp.Choice('conv_input_kernel', values=[3,5]),
                # activation
                activation = hp.Choice('conv_input_activation', ['relu','tanh']),
                padding="same"
              ))
    
    # adding convolutional layer
    for i in range (hp.Int("num_layers",1,6)):
        model.add(Conv1D(
                    # filters
                    filters = hp.Int(f'conv_{i}_filter', min_value=16, max_value=128, step=16),
                    # kernel_size
                    kernel_size = hp.Choice(f'conv_{i}_kernel', values=[3,5]),
                    # activation
                    activation = hp.Choice(f'conv_{i}_activation', ['relu','tanh']),
                    padding="same"
                ))
    
    # Tune whether to use dropout
    #if hp.Boolean("dropout"):
        #model.add(Dropout(rate=0.25))
    
    # pooling layer
    model.add(GlobalAveragePooling1D())
    
    # output layer
    model.add(Dense(8, activation='softmax'))
    
    # definr the optimizer learning rate as hyperparameter
    lr = hp.Float("lr", min_value=1e-3, max_value=1e-2, sampling="log")
    
    # compile model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy"])
    return model

In [ ]:
hp_cnn_cl = keras_tuner.HyperParameters()
build_cnn_cl_model(hp_cnn_cl)

tuner_cnn_cl = keras_tuner.RandomSearch(
    hypermodel=build_cnn_cl_model,
    objective="val_accuracy",
    max_trials=5,
    #executions_per_trial=3,
    overwrite=True,
    directory="mydir",
    project_name="test_cnn_cl",
)

tuner_cnn_cl.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val),
                 callbacks=[keras.callbacks.TensorBoard('my_dir/tb_cnn_cl')])

In [ ]:
models_cnn_cl = tuner_cnn_cl.get_best_models(num_models=2)
best_cnn_cl_model = models_cnn_cl[0]
best_cnn_cl_model.build(input_shape=(10,20))
best_cnn_cl_model.summary()

In [ ]:
best_cnn_cl_model.save('models_cnn_cl')

In [ ]:
test_cnn_cl_model = keras.models.load_model('models_cnn_cl')

plot_model(test_cnn_cl_model, show_shapes=True, show_layer_names=True)

In [ ]:
score, acc = test_cnn_cl_model.evaluate(X_test, y_test) #batch_size=batch_size
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
plt.plot(df0_0['cartForce1_X'],label="Fx")
plt.plot(df0_0['cartForce1_X'],label="Fy")
plt.plot(df0_0['cartForce1_X'],label="Fz")
plt.plot(df0_0['cartTorque1_TauX'],label="Tx")
plt.plot(df0_0['cartTorque1_TauX'],label="Ty")
plt.plot(df0_0['cartTorque1_TauX'],label="Tz")
for k in range(0,7):
    plt.plot(df0_0['axisTauExtMsr_LBR_iiwa_14_R820_1[{0}]'.format(k)],label="ext_j{0}".format(k+1))
for k in range(0,7):
    plt.plot(df0_0['axisTauMsr_LBR_iiwa_14_R820_1[{0}]'.format(k)],label="int_j{0}".format(k+1))
             
plt.legend()